<font size="5">**Algorytmy danych geoprzestrzennych**</font><br>
<font size="4">Wprowadzenie do PyQGIS</font>

<font size="4">Krzysztof Dyba</font>

PyQGIS to biblioteka języka Python udostępniająca interfejs programistyczny do aplikacji QGIS, dzięki czemu możliwa jest interakcja pomiędzy kodem napisanym w Pythonie, a środowiskiem i elementami QGIS (np. projekt, warstwy, funkcje, itd.). Biblioteka odgrywa kluczową rolę w automatyzacji zadań oraz tworzeniu niestandardowych narzędzi i wtyczek do rozszerzenia funkcjonalności QGIS. PyQGIS jest częścią projektu QGIS, który jest darmowy i otwarty.

Przykładowe zastosowania:
- Automatyzacja (przetwarzanie wsadowe, skrypty do czyszczenia i walidacji danych, generowanie map),
- Tworzenie nowwych narzędzi (wtyczki, specjalistyczne algorytmy geoprzetwarzania, niestandardowe interfejsy),
- Zaawansowane analizy przestrzenne (wykorzystanie zewnętrznych bibliotek, np. `numpy`, `scikit-learn`).

# Instalacja

Instalacja jest bardzo prosta -- należy pobrać program QGIS z oficjalnej strony https://qgis.org/download/ i zainstalować według wyświetlanych instrukcji. Środowisko Python i podstawowe biblioteki są automatycznie instalowane wraz z QGIS.

# Pierwsze kroki

Zasadniczo bibliotekę PyQGIS możemy wykorzystać na dwa sposoby:

1. Zintegrowana konsola Pythona w QGIS.

QGIS ma wbudowaną konsolę Pythona, w której można wykonywać skrypty interaktywnie. Aby ją włączyć kliknij `Wtyczki` > `Konsola Pythona` w pasku narzędzi. Można również wykorzystać skrót klawiszowy: `CTRL` + `ALT` + `P`. Oprócz konsoli, wbudowany jest bardzo prosty edytor kodu.

2. Samodzielne skrypty.

PyQGIS można również wykorzystać poza aplikacją QGIS w samodzielnych skryptach Pythona.

Sprawdź czy wszystko działa prawidłowo wykonując następujące polecenie:

```python
from qgis.core import *
```

## Dane wektorowe

### Wczytywanie

W folderze `dane` znajduje się plik *powiaty.gpkg*, który zawiera granice powiatów w wielkopolsce z [Państowego Rejestru Granic](https://www.geoportal.gov.pl/pl/dane/panstwowy-rejestr-granic-prg/). Żeby go wczytać należy wykorzystać klasę [QgsVectorLayer](https://qgis.org/pyqgis/master/core/QgsVectorLayer.html) i zdefiniować trzy argumenty:

1. ścieżkę do pliku,
2. nazwę warstwy pod jaką zostanie wyświetlony,
3. backend do wczytania danych (*data provider*), np. [ogr](https://gdal.org/en/stable/drivers/vector/index.html), [spatialite](https://www.gaia-gis.it/fossil/libspatialite/index), [postgres](https://www.postgresql.org/).

Ścieżkę do pliku można zdefiniować na dwa sposoby. Pierwszy to podanie **ścieżki bezwzględniej**, tj. wskazanie dokładnej lokalizacji, w której znajduje się plik. Na przykład:

```
sciezka = "C:\Users\Krzysztof\Desktop\dane.csv"
# lub sciezka = "C:\\Users\\Krzysztof\\Desktop\\dane.csv"
```

Nie jest to jednak rekomendowana metoda, ponieważ zakłada, że struktura katalogów pomiędzy różnymi komputerami i systemami jest identyczna. Drugi sposób polega na podaniu **ścieżki względnej**. W tym przypadku podajemy lokalizację pliku względem bieżącego katalogu roboczego lub projektu. Aby dowiedzieć się, gdzie znajduje się katalog roboczy, możemy użyć metody `getcwd()` z biblioteki `os`, a do jego zmiany metody `chdir()`. Uwaga: domyślna ścieżka katalogu roboczego w konsoli QGIS, a systemowej są różne!

In [1]:
import os
print(os.getcwd())

C:\Users\Krzysztof\Desktop


Zauważ również, że w zależności od systemu jest wykorzystywany różny separator katalogów w hierarchii. Aby prawidłowo wskazać ścieżkę do pliku należy wykorzystać metodę `os.path.join()`.

In [2]:
print(os.path.join("projekt", "dane", "pomiary.csv"))

projekt\dane\pomiary.csv


Spróbujmy teraz wczytać nasz plik.

In [3]:
from qgis.core import QgsVectorLayer

# określenie ścieżki do pliku
sciezka_do_pliku = os.path.join("algorytmy-geoprzestrzenne", "dane", "powiaty.gpkg")

# wczytanie warstwy wektorowej
wektor = QgsVectorLayer(sciezka_do_pliku, "powiaty", "ogr")

# sprawdzenie czy warstwa została załadowana prawidłowo
print(wektor.isValid())

True


Jak możemy zauważyć warstwa została wczytana prawidłowo. Następnie możemy wyświetlić ją w QGIS (pod warunkiem, że kod wykonujemy w QGIS).

```python
from qgis.core import QgsProject
QgsProject.instance().addMapLayer(wektor)
```

### Wyświetlanie metadanych

W kolejnym kroku możemy wyświetlić podstawowe informacje o warstwie, tj.:
- rodzaj geometrii,
- zakres przestrzenny,
- liczba obiektów oraz liczba atrybutów,
- układ współrzędnnych.

In [4]:
# rodzaj geometrii
rodzaj_geometrii = wektor.geometryType().name
print("Rodzaj geometrii:", rodzaj_geometrii)

# zakres przestrzenny
zakres = wektor.extent()
print("Zakres przestrzenny:", zakres.toString())

# liczba obiektów
liczba_obiekty = wektor.featureCount()
print("Liczba obiektów:", liczba_obiekty)

# liczba atrybutów
liczba_atrybuty = wektor.fields().count()
print("Liczba atrybutów:", liczba_atrybuty)

# układ współrzędnnych
uklad = wektor.crs()
print("CRS:", uklad.authid())

Rodzaj geometrii: Polygon
Zakres przestrzenny: 281949.3437500000000000,360241.7500000000000000 : 507167.3125000000000000,645513.6250000000000000
Liczba obiektów: 35
Liczba atrybutów: 5
CRS: EPSG:2180


### Zapisywanie

Dostępne są dwie funkcje:
1. [QgsVectorFileWriterV3](https://qgis.org/pyqgis/master/core/QgsVectorFileWriter.html)
2. [QgsVectorLayerExporter](https://qgis.org/pyqgis/master/core/QgsVectorLayerExporter.html)

In [5]:
from qgis.core import QgsVectorFileWriter

output_path = "test.json"
output_format = "GeoJSON"

error = QgsVectorFileWriter.writeAsVectorFormatV3(
    wektor,
    output_path
)

TypeError: QgsVectorFileWriter.writeAsVectorFormatV3(): not enough arguments

In [20]:
from qgis.core import QgsVectorLayerExporter

output_path = "test.json"
output_format = "GeoJSON"

exporter = QgsVectorLayerExporter(
    output_path,
    "ogr",
    geometryType = wektor.wkbType(),
    fields = wektor.fields(),
    crs = wektor.crs()
)

exporter.exportLayer(
    wektor,
    output_path,
    "ogr",
    wektor.crs()
)

(<VectorExportResult.ErrorInvalidLayer: 7>, 'Loading of layer failed')

## Dane rastrowe

### Wczytywanie

In [ ]:
from qgis.core import QgsRasterLayer

sciezka_do_pliku = os.path.join("algorytmy-geoprzestrzenne", "dane", "DEM.tif")
raster = QgsRasterLayer(sciezka_do_pliku, "DEM")
print(raster.isValid())

```python
from qgis.core import QgsProject
QgsProject.instance().addMapLayer(raster)
```

### Wyświetlanie metadanych

Metadane rastra:
- liczba wierszy, kolumn, kanałów,
- zakres przestrzenny,
- układ przestrzenny,
- rozdzielczość pikseli.

In [ ]:
print("Szerokość (Kolumny):", raster.width())
print("Wysokość (Wiersze):", raster.height())
print("Liczba kanałów:", raster.bandCount())
print("Zakres:", raster.extent().toString())
print("CRS:", raster.crs().authid())
print("Rozdzielczość (Rozmiar komórki):", raster.rasterUnitsPerPixelX(),
      raster.rasterUnitsPerPixelY())

Dla każdego kanału:
- nazwa kanału,
- brak wartości,
- typ danych,
- wartość maksymalna,
- wartość minimalna.

In [ ]:
for band in range(1, raster.bandCount() + 1):
    print("Nazwa kanału:", raster.bandName(band))
    print("Wartość NoData:", raster.dataProvider().sourceNoDataValue(band))
    print("Typ danych:", raster.dataProvider().dataType(band).name)
    print("Wartość maksymalna:", raster.dataProvider().bandStatistics(band).minimumValue)
    print("Wartość minimalna:", raster.dataProvider().bandStatistics(band).maximumValue)

### Zapisywanie

# Zadania

4. Napisz funkcję, która obliczy wielkość piksela.
5. Napisz funkcję, która ładnie zaprezentuje metadane warstwy wektorowej oraz rastrowej.

TODO:
1. Wczytanie warstwy wektorowej
  -- wczytanie tylko obiektow, ktore spelniaja warunek
2. Zapisanie warstwy wektorowej
3. Wyświetl podstawowe informacje (crs, liczba rekordow, typ geometrii, nazwy atrybutow)
4. Iterowanie po obiektach
5. Stworzenie nowej warstwy progrmistycznie
6. Reprojekcja warstwy
7. Obliczenie statystyk atrybytów
8. Intersekcja dwóch warstw wektorowych
9. Tworzenie buforów